# Data preparation

In [ ]:
import pandas as pd
from tqdm import tqdm_notebook, tqdm
from collections import defaultdict
import numpy as np
from Sparse_vector.sparse_vector import SparseVector
import os
from joblib import load, dump, Parallel, delayed

taget_antigen = ['DNase-seq', 'Histone', 'RNA polymerase', 'TFs and others']
chroms = [f'chr3']

In [ ]:
%%bash
mkdir ../data/mm9_dna
mkdir ../data/mm9_dna/raw
mkdir ../data/mm9_dna/sparse
cd ../data/mm9_dna/raw
wget http://hgdownload.cse.ucsc.edu/goldenPath/mm9/chromosomes/chr3.fa.gz
gzip -d chr3.fa.gz

In [ ]:
from Bio import SeqIO

for record in tqdm(SeqIO.parse("../data/mm9_dna/raw/chr3.fa", "fasta")):
    if record.id in chroms:
        dump(str(record.seq.upper()), f"../data/mm9_dna/sparse/{record.id}.pkl", 3)

# Sequences

In [ ]:
dna = {chrom:load(f'../data/mm9_dna/sparse/{chrom}.pkl') for chrom in tqdm(chroms)}
lens_of_chroms = {chrom: len(dna[chrom]) for chrom in dna}

In [ ]:
files = sorted(os.listdir('../data/mm9_features/raw/'))[::-1]

def sparser(file, lens_of_chroms, chroms):
#     print(file)
    loc_dd = {chrom:SparseVector(lens_of_chroms[chrom]) for chrom in chroms}
    df = pd.read_csv(f'../data/mm9_features/raw/{file}', header=None, sep='\t')
    for chrom, sub_df in tqdm(df.groupby(0), desc=file):
        if chrom not in chroms:
            continue
        vec = np.zeros(lens_of_chroms[chrom])
        for inter in sub_df.values:
            vec[inter[1]:inter[2]+1] = np.maximum(vec[inter[1]:inter[2]+1], inter[4])
        loc_dd[chrom] = SparseVector(vec)

    dump(loc_dd, f'../data/mm9_features/sparse/{file[:-4]}.pkl', 3)  

In [ ]:
Parallel(n_jobs = -1)(delayed(sparser)(file, lens_of_chroms, chroms) 
                                      for file in files if file.endswith('.bed'))

# ZDNA prepare

In [ ]:
def processor(file):
    loc_dd = {chrm:SparseVector(lens_of_chroms[chrm]) for chrm in chroms}
    df = pd.read_csv(f'../data/mm9_zdna/raw/{file}', sep='\t', header=None)
    for chrom, sub_df in tqdm(df.groupby(0)):
        if chrom not in chroms:
            continue
        vec = np.zeros(lens_of_chroms[chrom])
        for inter in sub_df.values:
            vec[inter[1]:inter[2]+1] = np.maximum(vec[inter[1]:inter[2]+1], 1)
        loc_dd[chrom] = SparseVector(vec)

    dump(loc_dd, f'../data/mm9_zdna/sparse/{file[:-4]}.pkl', 3)

In [ ]:
for file in os.listdir('../data/mm9_zdna/raw/'):
    if file.startswith("zdna_"):
        processor(file)

In [ ]:
loc_dd = {chrm:SparseVector(lens_of_chroms[chrm]) for chrm in chroms}
df = pd.read_csv('../data/mm9_zdna/raw/blacklist_mm9.bed', sep='\t', header=None, names=range(3), index_col=False)
for chrom, sub_df in tqdm(df.groupby(0)):
    if chrom not in chroms:
        continue
    vec = np.zeros(lens_of_chroms[chrom])
    for inter in sub_df.values:
        vec[inter[1]:inter[2]+1] = np.maximum(vec[inter[1]:inter[2]+1], 1)
    loc_dd[chrom] = SparseVector(vec)
dump(loc_dd, f'../data/mm9_zdna/sparse/blacklist_mm9.pkl', 3)